In [1]:


from vecstack import stacking
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")





In [2]:
trainfile = '/Users/anandvasudevan/Downloads/house-prices-advanced-regression-techniques/train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = '/Users/anandvasudevan/Downloads/house-prices-advanced-regression-techniques/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe



print(trainData.shape)
print(testData.shape)




(1460, 81)
(1459, 80)


# Data Preprocessing and Clean-up

In [3]:
trainData.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [4]:
testData.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [5]:
#Extract Target Column before doing missing value substitutions and one-hot encoding======
Target_Train_Cols = trainData["SalePrice"]#make copy of target column
trainData = trainData.drop(["SalePrice"], axis=1) #extracting training data without the target column

print(trainData.shape)

(1460, 80)


In [6]:
#DROP COLUMNS WITH LOTS OF MISSING VALUES===============================
#CAN ALSO DROP ROWS WITH LOTS OF MISSING VALUES
#Combine Train data and test data first so that the SAME COLUMNS are DROPPED in each
combined_Data = pd.concat([trainData, testData], keys=[0,1])

combined_Data.isnull().sum().sort_values(ascending=False)

#combined_Data.shape

PoolQC           2909
MiscFeature      2814
Alley            2721
Fence            2348
FireplaceQu      1420
                 ... 
CentralAir          0
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
SaleCondition       0
Length: 80, dtype: int64

In [7]:
#Define threshold for dropping columns
percent=int(0.6*(combined_Data.shape[0]))
print(percent)
#Drop columns that have less than "thresh" number of non_Nans
td1=combined_Data.dropna(thresh=percent,axis=1)
print(td1.shape)

1751
(2919, 75)


In [44]:
#look at what other columns have missing values
td1.isnull().sum().sort_values(ascending=False)

Id               0
HeatingQC        0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
CentralAir       0
Heating          0
MSSubClass       0
TotalBsmtSF      0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation  

In [9]:
#NOW IMPUTE MISSING VALUES FOR THE OTHER COLUMNS=========================
#IMPUTE (SUBSTITUTE) MEAN VALUES FOR NaN IN NUMERIC COLUMNS 
numeric=td1.select_dtypes(include=['int','float64']).columns
for num in numeric:
  td1[num]=td1[num].fillna(td1[num].mean())

#IMPUTE (SUBSTITUTE) MODE VALUES FOR NaN IN CATEGORICAL COLUMNS
train_cat_cols = td1.select_dtypes(exclude=['int','float64']).columns#selecting the categorical columns
for colss in train_cat_cols:
  if(td1.iloc[0][colss]=="N"):
        td1[colss]=td1[colss].fillna("N")
  else:
    td1[colss]=td1[colss].fillna(td1[colss].mode())
  
print(td1.head(20))

      Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0 0    1          60       RL    65.000000     8450   Pave      Reg   
  1    2          20       RL    80.000000     9600   Pave      Reg   
  2    3          60       RL    68.000000    11250   Pave      IR1   
  3    4          70       RL    60.000000     9550   Pave      IR1   
  4    5          60       RL    84.000000    14260   Pave      IR1   
  5    6          50       RL    85.000000    14115   Pave      IR1   
  6    7          20       RL    75.000000    10084   Pave      Reg   
  7    8          60       RL    69.305795    10382   Pave      IR1   
  8    9          50       RM    51.000000     6120   Pave      Reg   
  9   10         190       RL    50.000000     7420   Pave      Reg   
  10  11          20       RL    70.000000    11200   Pave      Reg   
  11  12          60       RL    85.000000    11924   Pave      IR1   
  12  13          20       RL    69.305795    12968   Pave      IR2   
  13  

In [10]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES 
td1.isnull().sum().sort_values(ascending=False)

GarageQual       159
GarageCond       159
GarageFinish     159
GarageType       157
BsmtCond          82
                ... 
BsmtUnfSF          0
MSSubClass         0
Heating            0
HeatingQC          0
SaleCondition      0
Length: 75, dtype: int64

In [11]:
td1=td1.drop(columns=['GarageFinish',
'GarageQual',
'GarageCond',
'GarageType',
'BsmtExposure',
'BsmtCond',
'BsmtQual',
'BsmtFinType2',
'BsmtFinType1',
'MasVnrType',
'MSZoning',
'Utilities',
'Functional',
'Exterior2nd',
'Exterior1st',
'KitchenQual',
'SaleType',
'Electrical'])

In [12]:
#DO ONE-HOT ENCODING ON CATEGORICAL VARIABLES==============================================
#The below function returns a list of categorical features which are not numeric. 
train_cat_cols = td1.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)

#If there are categorical columns which are encoded as numeric ones 
#then we need to explicitly enter the column names in a list and concatenate the two lists in python.
#ONE-HOT ENCODING-generate one-hot encoding on a common basis -THIS TAKES 30 MINS

combined_Data = pd.get_dummies(td1,train_cat_cols)
combined_Data.head(10)





(20,)
Index(['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition'],
      dtype='object')


Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0 0   1          60    65.000000     8450            7            5   
  1   2          20    80.000000     9600            6            8   
  2   3          60    68.000000    11250            7            5   
  3   4          70    60.000000     9550            7            5   
  4   5          60    84.000000    14260            8            5   
  5   6          50    85.000000    14115            5            5   
  6   7          20    75.000000    10084            8            5   
  7   8          60    69.305795    10382            7            6   
  8   9          50    51.000000     6120            7            5   
  9  10         190    50.000000     7420            5            6   

     YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  CentralAir_Y  \
0 0       2003          2003       196.0       706.0  ...             1   
  1       1976          1976         0.0       978.0  ...             1   
  2       2001          2002       162.0       486.0  ...             1   
  3       1915          1970         0.0       216.0  ...             1   
  4       2000          2000       350.0       655.0  ...             1   
  5       1993          1995         0.0       732.0  ...             1   
  6       2004          2005       186.0      1369.0  ...             1   
  7       1973          1973       240.0       859.0  ...             1   
  8       1931          1950         0.0         0.0  ...             1   
  9       1939          1950         0.0       851.0  ...             1   

     PavedDrive_N  PavedDrive_P  PavedDrive_Y  SaleCondition_Abnorml  \
0 0             0             0             1                      0   
  1             0             0             1                      0   
  2             0             0             1                      0   
  3             0             0             1                      1   
  4             0             0             1                      0   
  5             0             0             1                      0   
  6             0             0             1                      0   
  7             0             0             1                      0   
  8             0             0             1                      1   
  9             0             0             1                      0   

     SaleCondition_AdjLand  SaleCondition_Alloca  SaleCondition_Family  \
0 0                      0                     0                     0   
  1                      0                     0                     0   
  2                      0                     0                     0   
  3                      0                     0                     0   
  4                      0                     0                     0   
  5                      0                     0                     0   
  6                      0                     0                     0   
  7                      0                     0                     0   
  8                      0                     0                     0   
  9                      0                     0                     0   

     SaleCondition_Normal  SaleCondition_Partial  
0 0                     1                      0  
  1                     1                      0  
  2                     1                      0  
  3                     0                      0  
  4                     1                      0  
  5                     1                      0  
  6                     1                      0  
  7                     1                      0  
  8                     0                      0  
  9                     1                      0  

[10 rows x 161 columns]

In [13]:
#CHECK IF THERE ARE ANY REMAINING MISSING VALUES 
td1.isnull().sum().sort_values(ascending=False)

Id               0
HeatingQC        0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
CentralAir       0
Heating          0
MSSubClass       0
TotalBsmtSF      0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation  

In [14]:
#DO ONE-HOT ENCODING ON CATEGORICAL VARIABLES==============================================
#The below function returns a list of categorical features which are not numeric. 
train_cat_cols = td1.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)

#If there are categorical columns which are encoded as numeric ones 
#then we need to explicitly enter the column names in a list and concatenate the two lists in python.
#ONE-HOT ENCODING-generate one-hot encoding on a common basis -THIS TAKES 30 MINS

combined_Data = pd.get_dummies(td1,train_cat_cols)
combined_Data.shape




(20,)
Index(['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition'],
      dtype='object')


(2919, 161)

In [15]:
#Separate Train data and test data
train_Data = combined_Data.xs(0)
test_Data = combined_Data.xs(1)
print(train_Data.shape)
print(test_Data.shape)

train_Data=pd.concat([train_Data,Target_Train_Cols], axis=1)
print(train_Data.shape)

(1460, 161)
(1459, 161)
(1460, 162)


In [16]:
#Extract training and test data
y_train = train_Data["SalePrice"]
X_train = train_Data.drop(["SalePrice"], axis=1) #extracting training data without the target column
#y_test = testData["SalePrice"]
X_test = test_Data

print(X_train.shape)
X_test.shape



(1460, 161)


(1459, 161)

In [ ]:
a,b =Xtrain.columns, Xtest.columns
(a | b).difference(a & b)

In [49]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
clf_predict_Train=clf.predict(X_train)

#clf.feature_importances_
#mean_squared_error(Ytest,clf_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,clf_predict_Train)))
clf_predict_Test=clf.predict(X_test)


RMSE (training) for Decision Tree:  0.000000


In [51]:
clf = DecisionTreeRegressor()
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,5)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeRegressor(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#Obtain confusion matrix for the result above.
#print("Confusion Matrix after hypertuning for Decision Tree")
#print(confusion_matrix(y_test,clf_predict))

{'min_samples_split': 40, 'max_depth': 16}


In [53]:
#Save predictions
df_DT=pd.DataFrame()
df_DT['Id']=testData['Id']
df_DT['SalePrice']=clf_predict
export_csv = df_DT.to_csv('hpdt1.csv', index = None)


In [20]:
#Random Forest Regressor==============================================================================
#=================================================================================================

rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
rfc_predict_Train=rfc.predict(X_train)

mean_squared_error(y_train,rfc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(X_test)
#mean_squared_error(y_test,rfc_predict_Test)
#print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,rfc_predict_Test)))



RMSE (training) for Decision Tree:116670734.192372


In [41]:
#Save predictions
df_RF=pd.DataFrame()
df_RF['Id']=testData['Id']
df_RF['SalePrice']=rfc_predict_Test
export_csv = df_RF.to_csv('rf1.csv', index = None)

In [42]:
rfr= RandomForestRegressor()
rand_parameters={'min_samples_leaf' : range(20,200,10),'max_depth': 
            range(5,20,3),'max_features':['auto', 'sqrt'],'n_estimators':[30,40,50,60]}
rfr_random = RandomizedSearchCV(rfr,rand_parameters,n_iter=25,cv=5)
rfr_random.fit(S_Train, y_train)
random_parm_rfr=rfr_random.best_params_
print(random_parm_rfr)

#Construct Random Forest with best parameters
rfr= RandomForestRegressor(**random_parm_rfr)
rfr.fit(X_train,y_train)
rfr_predict_Test = rfr.predict(X_test)

{'n_estimators': 30, 'min_samples_leaf': 20, 'max_features': 'sqrt', 'max_depth': 14}


In [43]:
#Save predictions
df_RF2=pd.DataFrame()
df_RF2['Id']=testData['Id']
df_RF2['SalePrice']=rfr_predict_Test
export_csv = df_RF2.to_csv('rfhpt1.csv', index = None)

In [22]:
#Gradient Boosting Regressor================================================================================

abc =GradientBoostingRegressor()
abc.fit(X_train, y_train)
abc_predict_Train=abc.predict(X_train)

mean_squared_error(y_train,abc_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,abc_predict_Train)))
#abc_predict_Test=rfc.predict(X_test)
#mean_squared_error(y_test,abc_predict_Test)
#print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,abc_predict_Test)))




RMSE (training) for Decision Tree:238074063.580733


In [61]:
# HyperParameter Tuning for Gradient Boosting Regressor
abc =GradientBoostingRegressor()
search_grid={'min_samples_leaf' : range(20,200,10),'max_depth': 
            range(5,20,3),'max_features':['auto', 'sqrt'],'n_estimators':[30,40,50,60]}
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Run the model using parameters found above
abc= GradientBoostingRegressor(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)

{'n_estimators': 50, 'min_samples_leaf': 80, 'max_features': 'auto', 'max_depth': 14}


In [62]:
#Save predictions
df_abc=pd.DataFrame()
df_abc['Id']=testData['Id']
df_abc['SalePrice']=abc_predict
export_csv = df_abc.to_csv('HTGBR1.csv', index = None)

In [24]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingRegressor(), RandomForestRegressor(), DecisionTreeRegressor() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, y_train, X_test,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2, shuffle = True)



___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [17062.60959745]
    fold  1:  [16562.66939733]
    fold  2:  [15681.48811527]
    fold  3:  [14572.12943469]
    ----
    MEAN:     [15969.72413619] + [946.35269160]
    FULL:     [15969.72413619]

model  1:     [RandomForestRegressor]
    fold  0:  [17359.83205479]
    fold  1:  [18204.46432877]
    fold  2:  [17509.28205479]
    fold  3:  [16387.39813699]
    ----
    MEAN:     [17365.24414384] + [648.31503577]
    FULL:     [17365.24414384]

model  2:     [DecisionTreeRegressor]
    fold  0:  [26475.72328767]
    fold  1:  [24675.06301370]
    fold  2:  [28542.76438356]
    fold  3:  [22899.06301370]
    ----
    MEAN:     [25648.153

In [25]:
#STACKING - CONTRUCT A RANDOM FOREST MODEL==============================
model = RandomForestRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)





In [26]:
mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
#mean_squared_error(y_test,y_pred_test)
#print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:337207617.554559


In [55]:
#HYPER PARAMETER TUNING ON THE STACKED MODEL
rfr= RandomForestRegressor()
rand_parameters={'min_samples_leaf' : range(20,200,10),'max_depth': 
            range(5,20,3),'max_features':['auto', 'sqrt'],'n_estimators':[30,40,50,60]}
rfr_random = RandomizedSearchCV(rfr,rand_parameters,n_iter=25,cv=5)
rfr_random.fit(S_Train, y_train)
random_parm_rfr=rfr_random.best_params_
print(random_parm_rfr)

#Construct Random Forest with best parameters
rfr= RandomForestRegressor(**random_parm_rfr)
rfr.fit(X_train,y_train)
rfr_predict_Test = rfr.predict(X_test)

{'n_estimators': 30, 'min_samples_leaf': 20, 'max_features': 'sqrt', 'max_depth': 11}


In [60]:
#Save predictions
rf_stacking=pd.DataFrame()
rf_stacking['Id']=testData['Id']
rf_stacking['SalePrice']=rfr_predict_Test
export_csv = rf_stacking.to_csv('FINAL STACKED PRED.csv',index = None)